In [1]:
#load libraries
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize

In [2]:
#load orders json file
with open('orders.json') as f:
    orders_json = json.load(f)

In [3]:
#flatten json data
orders_df = json_normalize(orders_json)

In [4]:
#add date only column
orders_df['Date'] = pd.to_datetime(orders_df['created_at']).dt.date

In [5]:
#line items information
order_information = []
order_items_df = orders_df[['id','line_items']]

for i, row in order_items_df.iterrows():
    for order in row['line_items']:
        order_information.append([row['id'],order['id'],order['product_id'],order['product_sku'],
                                  order['product_name'],order['price']])
        
order_information_df = pd.DataFrame(order_information)
order_information_df.columns = ['id','product_order_id','product_id','product_sku',
                                'product_name','price']#add header   

In [6]:
# combine order and line items
orders_line_items_df = pd.merge(orders_df, order_information_df, how='inner',on = 'id')

In [7]:
#generate min and max date
max_date = orders_df['Date'].max()
min_date = orders_df['Date'].min()

In [8]:
#currency information
CURRENCY_US = "USD"
CURRENCY_CAD = "CAD"

In [9]:
#foreign exchange api url setup
url = "https://api.exchangeratesapi.io/history?start_at=" + str(min_date) + "&end_at=" + str(max_date) + "&base=" + CURRENCY_US + "&symbols=" + CURRENCY_CAD

exchange_information = []
response = requests.get(url)
if (response.status_code != 200):
    print("URL is not working")  
else:
    response = requests.get(url)
    exchange_rates_data = response.json()
    exchange_rates_cad = exchange_rates_data["rates"]
    for exchange_rate_cad in exchange_rates_cad: 
        exchange_information.append([exchange_rate_cad,exchange_rates_cad[exchange_rate_cad]["CAD"]])
    exchange_df = pd.DataFrame(exchange_information)
    exchange_df.columns = ['Date','CAD_RATE'] #add headers

In [10]:
exchange_df.sort_values(by=['Date'], inplace=True)

In [11]:
#Update exchange information since some days are missing - filled using previous and future exchange rates
exchange_update = []
daterange = pd.date_range(min_date, max_date)
for single_date in daterange:
    exchange_update.append(single_date.strftime("%Y-%m-%d"))
    exchange_update_df = pd.DataFrame(exchange_update)
    exchange_update_df.columns = ['Date'] #add header

exchange_update_df = pd.merge(exchange_update_df, exchange_df, how='left',on = 'Date')
exchange_update_df = exchange_update_df.fillna(method='ffill')   
exchange_update_df = exchange_update_df.fillna(method='bfill')

exchange_update_df['CAD_RATE'] = exchange_update_df['CAD_RATE'].round(decimals=2)

In [12]:
#combine exchange rate and line item orders
orders_line_items_df['Date'] = orders_line_items_df['Date'].astype(str)
orders_line_items_df.reset_index(drop=True, inplace=True)
exchange_update_df.reset_index(drop=True, inplace=True)
orders_line_items_final_df = pd.merge(orders_line_items_df, exchange_update_df, how='left',on = 'Date')

In [13]:
#generate canadian prices
orders_line_items_final_df['cad_price'] = orders_line_items_final_df['price']*orders_line_items_final_df['CAD_RATE']
orders_line_items_final_df['cad_price'] = orders_line_items_final_df['cad_price'].round(decimals=2)

In [14]:
#select information needed and rename columns
orders_line_items_CAD_final_df = orders_line_items_final_df[['id','customer.id','customer.name','customer.email','Date',
                                     'product_order_id','product_id','product_sku','product_name','price','cad_price']]
orders_line_items_CAD_final_df.columns = ['id','customer_id','customer_name','customer_email','date','product_order_id',
                               'product_id','product_sku','product_name','product_price','cad_price']

In [15]:
#Using sqllite to create db
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [16]:
connection = create_connection("order_app2.sqlite")

Connection to SQLite DB successful


In [17]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [18]:
#Create tables creation SQL
create_product_table = """
CREATE TABLE IF NOT EXISTS product (
  product_id INTEGER,
  product_name TEXT NOT NULL,
  product_sku TEXT NOT NULL,
  product_price real
  
);
"""

create_customer_table = """
CREATE TABLE IF NOT EXISTS customer (
  customer_id INTEGER PRIMARY KEY,
  customer_name TEXT,
  customer_email TEXT
);
"""

create_order_table = """
CREATE TABLE IF NOT EXISTS orders (
  order_id INTEGER PRIMARY KEY,
  order_date TEXT
);
"""

create_product_order_table = """
CREATE TABLE IF NOT EXISTS product_orders (
  product_order_id INTEGER PRIMARY KEY,
  order_id INTEGER,
  FOREIGN KEY (order_id) REFERENCES orders (order_id)
);
"""

create_customer_order_table = """
CREATE TABLE IF NOT EXISTS customer_orders (
  order_id INTEGER,
  customer_id INTEGER,
  PRIMARY KEY(order_id,order_id),
  FOREIGN KEY (order_id) REFERENCES orders (order_id),
  FOREIGN KEY (customer_id) REFERENCES customer (customer_id)
);
"""

In [19]:
#execute table generation queries
execute_query(connection, create_product_table)
execute_query(connection, create_customer_table)
execute_query(connection, create_order_table)
execute_query(connection, create_product_order_table)
execute_query(connection, create_customer_order_table)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [20]:
#load data
conn = connection
cur = conn.cursor()

#load product
products_df = orders_line_items_CAD_final_df[['product_id','product_sku',
                                              'product_name','product_price']].drop_duplicates()
products_df.to_sql('product', conn, if_exists='replace', index=False)

#load customer
customers_df = orders_line_items_CAD_final_df[['customer_id','customer_name',
                                              'customer_email']].drop_duplicates()
customers_df.to_sql('customer', conn, if_exists='replace', index=False)

#load orders
orders_df = orders_line_items_CAD_final_df[['id','date']].drop_duplicates()
orders_df.columns = ['order_id','order_date']
orders_df.to_sql('orders', conn, if_exists='replace', index=False)

#load product order
product_orders_df = orders_line_items_CAD_final_df[['product_order_id','id']].drop_duplicates()
product_orders_df.columns = ['product_order_id','order_id']
product_orders_df.to_sql('product_orders', conn, if_exists='replace', index=False)

#load customer order
product_orders_df = orders_line_items_CAD_final_df[['id','customer_id']].drop_duplicates()
product_orders_df.columns = ['order_id','customer_id']
product_orders_df.to_sql('customer_orders', conn, if_exists='replace', index=False)

conn.commit()
conn.close()